In [1]:
import os
import argparse

import easydict

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchmetrics.aggregation import MeanMetric
from torchmetrics.functional.classification import accuracy

from src.models import ConvNet
from src.engines import train, evaluate
from src.utils import load_checkpoint, save_checkpoint

# Jupyter 환경
args = easydict.EasyDict({
        "title" : "augmentation",
        "device" : "cuda",
        "root" : "data",
        "batch_size" : 128,
        "num_workers" : 2,
        "epochs" : 100,
        "lr" : 0.001,
        "weight_decay": 0.0001,
        "label_smoothing": 0.05,
        "drop_rate": 0.1,
        "logs": "logs",
        "checkpoints": "checkpoints",
        "resume": False
    })

def main(args):
    # Build dataset
    train_transform = T.Compose([
        T.ToTensor(),
        T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    train_data = CIFAR10(args.root, train=True, download=True, transform=train_transform)
    train_loader = DataLoader(train_data, args.batch_size, shuffle=True, num_workers=args.num_workers, drop_last=True)

    val_transform = T.Compose([
        T.ToTensor(),
        T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    val_data = CIFAR10(args.root, train=False, download=True, transform=val_transform)
    val_loader = DataLoader(val_data, batch_size=args.batch_size, num_workers=args.num_workers)

    # Build model
    model = ConvNet()
    model = model.to(args.device)

    # Build optimizer 
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    # Build scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs * len(train_loader))

    # Build loss function
    loss_fn = nn.CrossEntropyLoss()

    # Build metric function
    metric_fn = accuracy

    # Build logger
    train_logger = SummaryWriter(f'{args.logs}/train/{args.title}')
    val_logger = SummaryWriter(f'{args.logs}/val/{args.title}')

    # Load model
    start_epoch = 0
    if args.resume:
        start_epoch = load_checkpoint(args.checkpoints, args.title, model, optimizer)
    
    # Main loop
    for epoch in range(start_epoch, args.epochs):
        # train one epoch
        train_summary = train(train_loader, model, optimizer, scheduler, loss_fn, metric_fn, args.device)
        
        # evaluate one epoch
        val_summary = evaluate(val_loader, model, loss_fn, metric_fn, args.device)

        # write log
        train_logger.add_scalar('Loss', train_summary['loss'], epoch + 1)
        train_logger.add_scalar('Accuracy', train_summary['metric'], epoch + 1)
        val_logger.add_scalar('Loss', val_summary['loss'], epoch + 1)
        val_logger.add_scalar('Accuracy', val_summary['metric'], epoch + 1)

        # save model
        save_checkpoint(args.checkpoints, args.title, model, optimizer, epoch + 1)

In [2]:
!pip install torchsummary
from torchsummary import summary as summary

model = ConvNet()
model = model.to(args.device)
summary(model, (3,32,32)) # (model, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 16, 16]           2,688
              ReLU-2           [-1, 96, 16, 16]               0
            Conv2d-3          [-1, 192, 16, 16]         166,080
              ReLU-4          [-1, 192, 16, 16]               0
            Conv2d-5            [-1, 384, 8, 8]         663,936
              ReLU-6            [-1, 384, 8, 8]               0
            Conv2d-7            [-1, 384, 8, 8]       1,327,488
              ReLU-8            [-1, 384, 8, 8]               0
           Dropout-9                  [-1, 384]               0
           Linear-10                   [-1, 10]           3,850
Total params: 2,164,042
Trainable params: 2,164,042
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.88
Params size (MB): 8.26
Estimat

In [3]:
if __name__=="__main__":
    main(args)

Files already downloaded and verified
Files already downloaded and verified
tensor([[[[ 0.2848,  0.2073,  0.0328,  ..., -1.5761, -1.6537, -1.7118],
          [ 0.9439,  1.0021,  1.1571,  ..., -1.5180, -1.5567, -1.5761],
          [ 1.4673,  1.0602,  1.2541,  ..., -1.0527, -1.0527, -1.3047],
          ...,
          [-1.7118, -1.6343, -1.4404,  ..., -1.8475, -1.8669, -1.9057],
          [-1.8475, -1.7700, -1.6149,  ..., -1.8669, -1.8087, -1.8669],
          [-2.0608, -2.0026, -2.0414,  ..., -1.9832, -2.0220, -2.0608]],

         [[ 0.3941,  0.3351,  0.2761,  ..., -0.5499, -0.7662, -1.0809],
          [ 0.7874,  0.8071,  0.9841,  ..., -0.6286, -0.7662, -0.9432],
          [ 1.2398,  0.7284,  0.9054,  ..., -0.2942, -0.4712, -0.7466],
          ...,
          [-1.6316, -1.5136, -1.2776,  ..., -1.9856, -2.0053, -2.0643],
          [-1.9069, -1.7692, -1.5726,  ..., -1.9856, -1.9069, -1.9463],
          [-2.1429, -2.0446, -2.0053,  ..., -2.0446, -2.1036, -2.1429]],

         [[ 1.5831,  1.310

tensor([[[[-0.6069, -0.7813, -1.1690,  ..., -1.5180, -1.5180, -1.5180],
          [-0.9170, -0.8589, -1.0915,  ..., -1.5180, -1.5180, -1.5374],
          [-0.8977, -0.7813, -1.0334,  ..., -1.4986, -1.4986, -1.5374],
          ...,
          [-1.0334, -1.5180, -1.3629,  ..., -1.4404, -1.4598, -1.5374],
          [-0.8783, -1.0140, -1.2854,  ..., -1.4017, -1.4211, -1.4986],
          [-0.6844, -0.3161, -0.7232,  ..., -1.4017, -1.4211, -1.4792]],

         [[-0.7072, -0.6876, -0.7859,  ..., -1.1596, -1.1792, -1.1792],
          [-1.0219, -0.7466, -0.7072,  ..., -1.1792, -1.1792, -1.1989],
          [-1.0022, -0.6679, -0.6089,  ..., -1.1596, -1.1596, -1.1989],
          ...,
          [-1.1792, -1.4349, -1.3759,  ..., -1.1202, -1.1596, -1.2382],
          [-1.0809, -1.1006, -1.3956,  ..., -1.0809, -1.1202, -1.1989],
          [-0.8842, -0.5696, -0.9432,  ..., -1.0612, -1.1202, -1.1792]],

         [[-1.4800, -1.8897, -2.1824,  ..., -2.2019, -2.2214, -2.2214],
          [-1.7727, -2.0068, -

tensor([[[[ 6.3375e-01,  4.5929e-01,  5.7560e-01,  ...,  6.1437e-01,
            7.1129e-01,  7.3068e-01],
          [ 7.3068e-01,  6.9191e-01,  7.5006e-01,  ...,  6.7252e-01,
            8.8576e-01,  7.5006e-01],
          [ 7.6945e-01,  9.2452e-01,  6.1437e-01,  ...,  8.2760e-01,
            8.0822e-01,  7.1129e-01],
          ...,
          [ 5.3683e-01,  5.7560e-01,  7.5006e-01,  ...,  1.1184e+00,
            1.2347e+00,  1.1765e+00],
          [ 6.3375e-01,  5.3683e-01,  7.1129e-01,  ...,  1.1184e+00,
            1.4673e+00,  1.0408e+00],
          [ 6.7252e-01,  6.7252e-01,  7.6945e-01,  ...,  1.2153e+00,
            1.0990e+00,  8.4699e-01]],

         [[ 5.7111e-01,  3.9410e-01,  4.9244e-01,  ...,  5.5144e-01,
            6.4977e-01,  6.3011e-01],
          [ 6.3011e-01,  5.9077e-01,  6.6944e-01,  ...,  5.9077e-01,
            8.2677e-01,  6.4977e-01],
          [ 6.8911e-01,  8.4644e-01,  5.3177e-01,  ...,  7.0877e-01,
            6.8911e-01,  5.7111e-01],
          ...,
     

tensor([[[[ 0.7113,  0.5562,  0.6144,  ...,  0.6144,  0.6144,  0.6144],
          [ 0.8470,  0.6919,  0.7694,  ...,  0.6531,  0.6531,  0.6338],
          [ 0.7501,  0.6144,  0.6919,  ...,  0.6919,  0.6919,  0.6531],
          ...,
          [-2.1189, -2.1577, -2.0220,  ..., -1.1497, -1.1109, -1.1884],
          [-2.1189, -2.2158, -2.2352,  ..., -1.2272, -1.0915, -1.1690],
          [-2.1577, -2.3515, -2.3128,  ..., -1.4211, -1.2854, -1.3047]],

         [[ 1.2594,  1.2201,  1.2398,  ...,  1.2201,  1.2594,  1.1808],
          [ 1.3184,  1.2988,  1.3184,  ...,  1.2988,  1.3184,  1.2594],
          [ 1.3578,  1.3381,  1.3971,  ...,  1.3184,  1.3184,  1.2398],
          ...,
          [-1.5726, -1.4742, -1.3956,  ..., -1.0612, -1.0809, -1.1596],
          [-1.6119, -1.5529, -1.6512,  ..., -1.1399, -1.0809, -1.1399],
          [-1.7496, -1.7692, -1.7889,  ..., -1.3169, -1.2186, -1.2382]],

         [[ 2.1099,  2.0904,  1.9928,  ...,  2.0123,  1.9928,  1.9928],
          [ 2.0904,  2.0709,  

tensor([[[[-9.3643e-01, -9.5581e-01, -9.1704e-01,  ..., -1.1109e+00,
           -1.3047e+00, -1.4986e+00],
          [-1.0527e+00, -1.1303e+00, -1.0915e+00,  ..., -1.3241e+00,
           -1.5761e+00, -1.7312e+00],
          [-1.3629e+00, -1.4211e+00, -1.3047e+00,  ..., -1.2272e+00,
           -1.4017e+00, -1.4986e+00],
          ...,
          [-1.2078e+00, -1.2272e+00, -1.1303e+00,  ..., -1.2078e+00,
           -1.3047e+00, -1.3629e+00],
          [-7.0381e-01, -6.6504e-01, -7.0381e-01,  ..., -1.1690e+00,
           -1.1109e+00, -1.1497e+00],
          [-4.3242e-01, -3.5488e-01, -6.4565e-01,  ..., -1.0915e+00,
           -9.3643e-01, -9.3643e-01]],

         [[-1.9590e-01, -2.1557e-01, -1.7623e-01,  ..., -3.7290e-01,
           -5.6957e-01, -7.6624e-01],
          [-3.1390e-01, -4.1224e-01, -3.5324e-01,  ..., -5.8924e-01,
           -8.4491e-01, -1.0022e+00],
          [-6.2857e-01, -6.8757e-01, -5.6957e-01,  ..., -4.9090e-01,
           -6.6791e-01, -7.6624e-01],
          ...,
     

...

tensor([[[[-2.1383, -1.9638, -1.4792,  ..., -2.2158, -2.3515, -2.3128],
          [-1.7118, -1.5955, -1.3629,  ..., -2.2158, -2.2934, -2.1771],
          [-1.6924, -1.5761, -1.7894,  ..., -1.5955, -1.9444, -1.6924],
          ...,
          [-0.0253,  0.0328,  0.0328,  ...,  0.3430,  0.2073,  0.0328],
          [-0.1029, -0.1223, -0.1610,  ..., -0.0253, -0.1610, -0.1998],
          [-0.3549, -0.1416, -0.3936,  ..., -0.0835, -0.1610, -0.4712]],

         [[-2.0053, -1.9463, -1.3956,  ..., -2.1036, -2.3199, -2.2806],
          [-1.4742, -1.3562, -1.0416,  ..., -2.0643, -2.2019, -2.1036],
          [-1.3759, -1.2186, -1.4349,  ..., -1.4939, -1.7889, -1.5332],
          ...,
          [ 0.5318,  0.5711,  0.5908,  ...,  0.9054,  0.7678,  0.6498],
          [ 0.4138,  0.3941,  0.3548,  ...,  0.5318,  0.3941,  0.3941],
          [ 0.1581,  0.3744,  0.1384,  ...,  0.4728,  0.3941,  0.0598]],

         [[-2.1238, -2.0263, -1.5971,  ..., -2.1238, -2.2214, -2.1824],
          [-1.8117, -1.7141, -

tensor([[[[-0.6263, -0.6263, -0.5100,  ...,  2.4559,  2.4559,  2.4559],
          [-0.9558, -0.9558, -0.7038,  ...,  2.4753,  2.4753,  2.4753],
          [-0.8395, -0.9558, -0.5875,  ...,  2.4753,  2.4559,  2.4559],
          ...,
          [ 1.8550,  1.0796, -1.4017,  ..., -1.5374, -1.5761, -1.5761],
          [ 1.6612,  1.5836,  0.3817,  ..., -1.0140, -1.0915, -1.3047],
          [ 2.0295,  2.0295,  1.9132,  ..., -1.2272, -1.0334, -0.9558]],

         [[-0.3532, -0.3532, -0.2352,  ...,  2.5378,  2.5378,  2.5378],
          [-0.7662, -0.7269, -0.4516,  ...,  2.5575,  2.5378,  2.5575],
          [-0.7072, -0.7859, -0.3729,  ...,  2.5575,  2.5378,  2.5378],
          ...,
          [ 1.9478,  1.0824, -1.4546,  ..., -1.3562, -1.3169, -1.2186],
          [ 1.6921,  1.4758,  0.2564,  ..., -0.7859, -0.7466, -0.8056],
          [ 2.0068,  1.9085,  1.8298,  ..., -0.9629, -0.7072, -0.4319]],

         [[-0.5630, -0.5825, -0.4655,  ...,  2.6952,  2.6952,  2.6952],
          [-0.7971, -0.9142, -

tensor([[[[ 3.6236e-01,  3.2359e-01,  3.2359e-01,  ..., -5.2934e-01,
           -5.6811e-01, -6.0688e-01],
          [ 3.4298e-01,  2.8482e-01,  2.6544e-01,  ..., -4.9057e-01,
           -5.2934e-01, -5.8750e-01],
          [ 3.0421e-01,  2.2667e-01,  2.2667e-01,  ..., -4.9057e-01,
           -5.0996e-01, -5.6811e-01],
          ...,
          [-1.0288e-01, -2.1919e-01, -1.4165e-01,  ..., -5.8750e-01,
           -6.2627e-01, -6.4565e-01],
          [-2.5796e-01, -3.3549e-01, -1.8042e-01,  ..., -7.2319e-01,
           -7.4258e-01, -7.4258e-01],
          [-4.4721e-02, -4.4721e-02,  3.2819e-02,  ..., -7.0381e-01,
           -7.2319e-01, -7.2319e-01]],

         [[ 4.7277e-01,  4.3344e-01,  4.3344e-01,  ..., -3.5324e-01,
           -3.9257e-01, -4.3190e-01],
          [ 4.5310e-01,  3.9410e-01,  3.7444e-01,  ..., -2.9424e-01,
           -3.3357e-01, -3.7290e-01],
          [ 4.1377e-01,  3.3510e-01,  3.3510e-01,  ..., -2.5490e-01,
           -2.7457e-01, -3.3357e-01],
          ...,
     

tensor([[[[-1.9444e+00, -1.3629e+00,  1.6851e-01,  ..., -2.1771e+00,
           -2.0608e+00, -2.0414e+00],
          [-2.1189e+00, -1.4211e+00,  9.2452e-01,  ..., -1.8863e+00,
           -1.9444e+00, -1.9444e+00],
          [-2.1189e+00, -1.6343e+00,  7.5006e-01,  ..., -1.9444e+00,
           -2.1383e+00, -2.1189e+00],
          ...,
          [-2.3515e+00, -2.2546e+00, -2.0801e+00,  ..., -1.6149e+00,
           -1.5955e+00, -1.4986e+00],
          [-2.3709e+00, -2.2934e+00, -2.1771e+00,  ..., -1.6731e+00,
           -1.6924e+00, -1.5180e+00],
          [-2.3709e+00, -2.3321e+00, -2.2934e+00,  ..., -1.7118e+00,
           -1.7312e+00, -1.6537e+00]],

         [[-1.9659e+00, -1.3759e+00, -2.5490e-01,  ..., -2.1036e+00,
           -1.9463e+00, -1.9069e+00],
          [-2.1036e+00, -1.4546e+00,  3.3510e-01,  ..., -1.9463e+00,
           -1.9266e+00, -1.8873e+00],
          [-2.0643e+00, -1.6709e+00, -1.8900e-02,  ..., -1.9659e+00,
           -2.1036e+00, -2.0839e+00],
          ...,
     

KeyboardInterrupt: 